# Cavities

This notebook illustrates some tools for working with protein cavities.

[Descriptive Documentation](https://downloads.ccdc.cam.ac.uk/documentation/API/descriptive_docs/cavities.html)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
%run ../Discovery_Notebook_utils.py

In [ ]:
from functools import reduce

In [ ]:
from ccdc.protein import Protein

from ccdc.cavity import Cavity, CavityDatabase

### Config

Directory containing protein structures...

In [ ]:
protein_dir = Path('proteins')

Cavity API feature types, ordered for convenience when inspecting cavities...

In [ ]:
feature_types = ['donor', 'acceptor',  'donor_acceptor', 'aromatic', 'pi', 'aliphatic', 'metal']

assert sorted(feature_types) == sorted(Cavity.feature_types.keys())  # Check all API feature types are accounted for

Local utility to depict a molecule with atoms labelled and it's identifier as a header...

In [ ]:
def mol2html(mol, hetero_only=True):
    
    atoms_to_label = []

    for n, atom in enumerate(mol.atoms, 1):

        if hetero_only and atom.atomic_symbol in ('C', 'H'): continue

        atoms_to_label.append(atom)
                        
    image = diagram_generator.image(mol, label_atoms=atoms_to_label)
    
    return HTML(f'<h3>{ligand.identifier}</h3>' + image)

### Initialization

In [ ]:
print(script_info)

In [ ]:
pd.options.display.max_rows = 1000  # Show more rows in JupyterLab

### Load a protein and find the cavities on it's surface

We will use the Factor Xa structure [1fax](https://www.ebi.ac.uk/pdbe/entry/pdb/1fax) in the examples below. Note that, for historical reasons, the Cavity API currently only reads PDB files. 

In [ ]:
pdb_code = '1fax'

In [ ]:
pdb_file = str(protein_dir / f'pdb{pdb_code}.ent')

In [ ]:
cavities = Cavity.from_pdb_file(str(protein_dir / f'pdb{pdb_code}.ent'))

len(cavities)

Make a table of cavity data for inspection...

In [ ]:
cavities_row = lambda x: (
    x.identifier,
    len(x.features),
    x.volume,
    len(x.ligand_identifiers),
    ', '.join(x.ligand_identifiers) if x.ligand_identifiers else '',
)

cavities_df = (
    pd.DataFrame(
        data=[cavities_row(x) for x in cavities],
        columns=['identifier', 'n_features', 'volume', 'n_ligands', 'ligand_identifiers']
    )
)

cavities_df

We will take the cavity containing a ligand...

In [ ]:
cavity = cavities[cavities_df.query("n_ligands > 0").iloc[0].name]

Make a table of cavity feature data for inspection...

In [ ]:
v2s = lambda x: '(' + ', '.join(f'{y:.3f}' for y in x) + ')'  # vector to string

features_row = lambda x: (
    x.residue.identifier,
    x.atom.label if x.atom else '',
    x.type,
    round(x.burial, 3),
    len(x.surface_points),
    v2s(x.coordinates),
    v2s(x.protein_vector),
    v2s(x.surface_vector),
)

features_df = (
    pd.DataFrame(
        data=[features_row(x) for x in cavity.features],
        columns=['residue', 'atom', 'type', 'burial', 'n_surface_points', 'coordinates', 'protein_vector', 'surface_vector']
    )
    .assign(type = lambda df: pd.Categorical(df['type'], ordered=True, categories=feature_types))  # Make type an ordered categorical column for display purposes
)

features_df.sort_values('type')  

### Cavity-ligand interactions

Get the ligand molecule...

In [ ]:
assert len(cavity.ligands) == 1  # There is only on ligand in the cavity

ligand = cavity.ligands[0]

ligand

In [ ]:
mol2html(ligand)

Make table of ligand-cavity interactions for inspection...

In [ ]:
def interactions_rows(atom):
    
    max_distance = 3.5

    if not (atom.is_donor or atom.is_acceptor): return []
        
    nearby_features = cavity.features_by_distance(atom.coordinates, max_distance)
    
    if not nearby_features: return []
    
    rows = []

    for type_ in ['donor', 'acceptor', 'donor_acceptor']:

        for feature in nearby_features & cavity.features_by_type(type_):

            feature_residue = feature.residue.identifier
            
            feature_atom = feature.atom.label
            
            distance = round(feature.point_distance(atom.coordinates), 2)

            rows.append([atom.label, atom.is_donor, atom.is_acceptor, feature_residue, feature_atom, type_, distance])
                
    return rows

interactions_df = pd.DataFrame(
    data=[row for rows in [interactions_rows(atom) for atom in ligand.atoms] for row in rows],
    columns=['ligand_atom', 'is_donor', 'is_acceptor', 'feature_residue', 'feature_atom', 'feature_type', 'distance']
)

interactions_df

### Features by Residue(s)

Here we examine the features associated with a list of residues. We will need to create an API [Protein](https://downloads.ccdc.cam.ac.uk/documentation/API/descriptive_docs/protein.html) object to work with.

In [ ]:
identifiers = ['A:ASP189', 'A:ALA190']

In [ ]:
protein = Protein.from_file(pdb_file)  # Protein object

In [ ]:
residues = [x for x in protein.residues if x.identifier in identifiers]  # Protein.Residue objects

Get features associated with the residues, then build a table of feature data for inspection...

In [ ]:
features_by_residues = cavity.features_by_residues(residues)

In [ ]:
(
    pd.DataFrame(
        data=[features_row(x) for x in features_by_residues],
        columns=['residue', 'atom', 'type', 'burial', 'n_surface_points', 'coordinates', 'protein_vector', 'surface_vector']
    )
    .assign(type = lambda df: pd.Categorical(df['type'], ordered=True, categories=['donor', 'acceptor',  'donor_acceptor', 'aromatic', 'pi', 'aliphatic', 'metal']))
)

### Visualization of Cavities

The API may be used to write a PyMOL script to set up a visualization of a cavity...

In [ ]:
# cavity.to_pymol_file('cavity.py')

In [ ]:
# pymol = start_pymol()

# if pymol:

#     pymol.load(str(pdb_file))
        
#     pymol.do('run cavity.py')

Similarly, the API may be used to write a `rlbcoor` file that allows visualization of a cavity...

In [ ]:
cavity.write('cavity.rlbcoor')

status = subprocess.Popen([hermes_exe, 'cavity.rlbcoor'])

### Comparison of Cavities

There are three methods in the API for comparing cavities, offering various trade-off of speed, accuracy and the diagnostic information returned.

We will illustrate these using the FXa structure used above, [1fax](https://www.ebi.ac.uk/pdbe/entry/pdb/1fax), and the Thrombin structure [5af9](https://www.ebi.ac.uk/pdbe/entry/pdb/5af9).

In [ ]:
pdb_codes = ['1fax', '5af9']  # '1ett'

In [ ]:
pdb_files = [str(protein_dir / f'pdb{pdb_code}.ent') for pdb_code in pdb_codes]

To keep things simple, for each protein we just take the first cavity that contains a ligand. In the present case, this gives us the cavities corresponding to the active sites, which is what we want. In general, of course, this would most likely be too simplistic; there might be no ligand present in that active site and/or there could be multiple binding sites (_e.g._ for inorganic ions, DMSO, GOL _etc._). Thus, in practice, a more sophisticated approach would be necessary.

In [ ]:
cavities = [[cavity for cavity in Cavity.from_pdb_file(pdb_file) if cavity.ligands][0] for pdb_file in pdb_files]  # Take first cavity with a ligand

Create a table of cavity data for inspection...

In [ ]:
cavities_df = pd.DataFrame(
    data=[cavities_row(x) for x in cavities],
    columns=['identifier', 'n_features', 'volume', 'n_ligands', 'ligand_identifiers']
)

cavities_df

The three methods are illustarted below, showing timings and the diagnostic information returned. Note that the scores returned for the various methods are not on  the same scale, so are not comparable between methods.

#### 1) Fast Cavity Graph Comparison

This is currently the default comparison method. It is derived from the original CavBase method (see below), but enhanced to provide faster comparisons with comparable accuracy.
<br>For details, see: “Extended Graph-Based Models for Enhanced Similarity Search in Cavbase”, T. Krotzky _et al_, IEEE/ACM Trans. Comput. Biol. Bioinf. 11, 878-890, 2014. DOI: [10.1109/TCBB.2014.2325020](https://dx.doi.org/10.1109/TCBB.2014.2325020).

In [ ]:
%%time

fast = cavities[0].compare(cavities[1], comparison_method=Cavity.ComparisonMethod.FAST_CAVITY_GRAPH_COMPARISON)

In [ ]:
print(f"""
Score:                {fast.score:.3f}
Largest clique size:  {fast.largest_clique_size}
Product graph size:   {fast.product_graph_size}
""")

#### 2) The original CavBase algorithm

This is implementation of the original CavBase cavity-graph comparison method. Although significantly slower than the default method, it does give access to the transformation matrix.
<br>For details, see: “A New Method to Detect Related Function Among Proteins Independent of Sequence and Fold Homology”, S. Schmitt _et al_, J. Mol. Biol. 323, 387-406, 2002.  DOI: [10.1016/S0022-2836(02)00811-2](https://dx.doi.org/10.1016/S0022-2836(02)00811-2).

In [ ]:
%%time

original = cavities[0].compare(cavities[1], comparison_method=Cavity.ComparisonMethod.CAVITY_GRAPH_COMPARISON)

In [ ]:
print(f"""
Score:       {original.score:.3f}
RMSD:        {original.rmsd:.3f}
Clique RMSD: {original.clique_rmsd:.3f}
N cliques:   {original.n_cliques}
N matches:   {original.n_matches}

Matrix:      {original.transformation_matrix}
""")

#### 3) Cavity histograms 

This is an implementation of the RAPMAD ('RApid Pocket MAtching using Distances') method. It is fast but provides less diagnostic information than the default method.
<br>For details, see: “Large-Scale Mining for Similar Protein Binding Pockets: With RAPMAD Retrieval on the Fly Becomes Real”, T. Krotzky _et al_, J. Chem. Inf. Model., 55, 165-179, 2015.  DOI: [10.1021/ci5005898](https://dx.doi.org/10.1021/ci5005898).

In [ ]:
%%time

histogram = cavities[0].compare(cavities[1], comparison_method=Cavity.ComparisonMethod.CAVITY_HISTOGRAMS_COMPARISON)

Note that *only* the score is returned...

In [ ]:
print(f"Score: {histogram:.3f}")  

### Overlaying Proteins

The API allows us to overlay proteins both _via_ a sequence alignment and _via_ the cavity representation.  We will illustrate both methods using the FXa and Thrombin structures used above.
Note that the first protein is the static template onto which the second is overlaid, so, here, (mobile) Thrombin is overlaid onto (static) FXa.

In [ ]:
pdb_codes

#### 1) Overlay _via_ sequence alignment around binding site

Proteins may be overlaid based on an alignment of the full sequences or, as here, just the binding sites; see Protein API [Descriptive Documentation](https://downloads.ccdc.cam.ac.uk/documentation/API/descriptive_docs/protein.html?highlight=chainsuperposition#superposing-protein-chains-and-binding-sites) and 
[API Documentation](https://downloads.ccdc.cam.ac.uk/documentation/API/modules/protein_api.html#ccdc.protein.Protein.ChainSuperposition) for further details.

For simplicity, we are simply using the first ligand of the template protein to define the binding site (which we are assuming is the active site of the protein). In practice, there may be no ligand and/or multiple binding sites other than the active site (_e.g._ for inorganic ions, DMSO, GOL _etc._) so care would need to be taken that the correct one was chosen. 

In [ ]:
proteins = [Protein.from_file(pdb_file) for pdb_file in pdb_files]  # Instantiate Protein objects

In [ ]:
superposer = Protein.ChainSuperposition()

binding_site = Protein.BindingSiteFromMolecule(proteins[0], proteins[0].ligands[0], 6.0)  # First ligand binding site of template protein

overlaid_protein = proteins[1].copy()  # The overlaid protein object will be altered, so we use a copy

rmsd, _ = superposer.superpose(proteins[0].chains[0], overlaid_protein.chains[0], binding_site1=binding_site)  # Use first chain

round(rmsd, 3)

Write out overlaid protein for visualization...

In [ ]:
with MoleculeWriter('overlaid_via_sequence.pdb') as writer:

    writer.write(overlaid_protein)

#### 2) Overlay _via_ cavities

For cavity-based overlay, we must use the original Cavity Graph Comparison method, as only it provides the transformation matrix. We will thus reuse the comparison result object `original` generated above.

In [ ]:
type(original)

In [ ]:
# For reference, the command used to generate the comprison object above was...

# original = cavities[0].compare(cavities[1], comparison_method=Cavity.ComparisonMethod.CAVITY_GRAPH_COMPARISON)  

In [ ]:
print(f"""
score:       {original.score:.3f}
rmsd:        {original.rmsd:.3f}
matrix:      {original.transformation_matrix}
""")

Apply the transformation...

In [ ]:
overlaid_protein = proteins[1].copy()  # The overlaid protein object will be altered, so we use a copy

overlaid_protein.transform(original.transformation_matrix)  

In [ ]:
type(overlaid_protein)

In [ ]:
overlaid_protein

Write out overlaid protein for visualization...

In [ ]:
with MoleculeWriter('overlaid_via_cavities.pdb') as writer:
    
    writer.write(overlaid_protein)

#### Visualize overlays

The overlays are very similar in this case, which is not surprising as the sequence homology is high and the folds are thus very similar. Where the cavity-overlay method could really help is in cases where homology is low but the cavities are similar as, for example, they have evolved to bind similar ligands.

In PyMOL...

In [ ]:
# pymol = start_pymol()

# if pymol:

#     pymol.load(pdb_files[0])  # Template protein

#     pymol.load('overlaid_via_sequence.pdb')
#     pymol.load('overlaid_via_cavities.pdb')

In Hermes...

In [ ]:
status = subprocess.Popen([hermes_exe, pdb_files[0], 'overlaid_via_sequence.pdb', 'overlaid_via_cavities.pdb'])

## Cavity Databases

This part of the tutorial is still under development, and more detail is being added. Please see the [Descriptive Documentation](https://downloads.ccdc.cam.ac.uk/documentation/API/descriptive_docs/cavities.html#cavity-databases) for more material.

A cavity database is built from a set of PDB files. For out test database, a small number of protease and kinase structures will be used.
As well as the human FXa (1fax) and Thrombin (5af9) structures used above, there is a bovine Thrombin structure (1ett), SARS-Cov-2 (COVID-19) and SARS-Cov MPro structures (6lu7 and 2amq) and four Human Spleen Tyrosine Kinase (SYK) structures (4px6, 4xg8, 4yjp and 5lma). 

Links to the PDBe are provided below so the entries used to build our test may be inspected....

In [ ]:
url = 'https://www.ebi.ac.uk/pdbe/entry/pdb/{pdb_code}'

pdb_codes = [x.name[3:-4] for x in protein_dir.glob('pdb*.ent')]

HTML('<ul>' + ''.join([f'<li> <a target="_blank" href="{url.format(pdb_code=x)}">{x}</a> </li>' for x in pdb_codes]) + '</ul>')

Create a fresh database...

In [ ]:
db_file = Path('cavities.db')

In [ ]:
if db_file.exists():
    
    os.remove(db_file)

In [ ]:
db = CavityDatabase(str(db_file))

In [ ]:
db.populate_all(protein_dir, verbose=True)

In [ ]:
print(f'''
Number of cavities: {db.get_number_of_cavities()}
Number of ligands:  {db.get_number_of_ligands()}
''')

In [ ]:
db_df = pd.DataFrame([{'name': name, **db.get_info_for_cavity(name)} for name in [x.identifier for x in db.cavities()]])

db_df

The cavity database is currently an SQLite database, and can be accessed as any other database could be. However, this may not be true in future and should not be relied upon.

In [ ]:
# import sqlite3

# conn = sqlite3.connect(db_file)

# pd.read_sql_query('SELECT a.name, b.* FROM cav_xml a JOIN cav_info b ON a.cavity_id = b.cavity_id', conn)

Cavities may be found by PDB code...

In [ ]:
cavities = db.get_cavities_by_pdb_code('1fax')

In [ ]:
[x.identifier for x in cavities]

Cavities may be also be found by ligand names...

In [ ]:
ligand_name = db.get_ligands_by_pdb_code('6lu7')[0]

ligand_name

In [ ]:
[x.identifier for x in db.get_cavities_by_ligand(ligand_name)]

More complex quereis may be set up using the database [Settings](https://downloads.ccdc.cam.ac.uk/documentation/API/modules/cavities_api.html#ccdc.cavity.CavityDatabase.Settings) object... 

In [ ]:
settings = db.Settings()

settings.aromatic_range = [0, 2]

settings.ligand_range = [1]

cavities = db.search(settings=settings)

[x.identifier for x in cavities]

The various cavity-comparion methods explore above may be used to search the database...

In [ ]:
# Give the methods friendly names...

methods = {
    'original':  Cavity.ComparisonMethod.CAVITY_GRAPH_COMPARISON,
    'fast':      Cavity.ComparisonMethod.FAST_CAVITY_GRAPH_COMPARISON,
    'histogram': Cavity.ComparisonMethod.CAVITY_HISTOGRAMS_COMPARISON,
}

In [ ]:
query = db.cavity('pdb1fax.1')

Search the test database using all methods...

In [ ]:
results = {name: db.search(query, comparison_method=value) for name, value in methods.items()}

Build a dataframe of the scores for inspection...

In [ ]:
dfs = [pd.DataFrame(data, columns=[method, 'cavity']).assign(**{method: lambda df: df[method] / df[method].iloc[0]}) for method, data in results.items()]

results_df = reduce(pd.merge, dfs)[['cavity', *results.keys()]]

In [ ]:
# results_df.to_pickle('results_df.pkl')
# results_df = pd.read_pickle('results_df.pkl')

In [ ]:
results_df

We can plot the scores for the various methods agains eaach other to see how the methods compare...

In [ ]:
scale = scale=altair.Scale(domain=(0, 1.1))

cols = results_df.columns.to_list()

In [ ]:
(
    altair.Chart(results_df).mark_circle(size=60, color='red').encode(
        x=altair.X('original', scale=scale),
        y=altair.Y('fast', scale=scale),
        tooltip=cols
    ).interactive() +
    altair.Chart(results_df).mark_circle(size=60, color='blue').encode(
        x=altair.X('original', scale=scale),
        y=altair.Y('histogram', scale=scale),
        tooltip=cols
    ).interactive()
)

Recall that the query is the active site for human Factor Xa. The human and bovine Thrombin active sites are clearly ranked highest by the `original` method, with the two MPro active sites being next and other protease cavities and the kinase active sites ranked below them.  The `fast` method also finds the protease active sites although the difference in scores is not pronounced at the lower end. The histogram method is not performing well on this set.

Search for a kinase active site...

In [ ]:
query_2 = db.get_cavities_by_pdb_code('5lma')[0]

query_2.identifier

In [ ]:
results_2 = {name: db.search(query_2, comparison_method=value) for name, value in methods.items()}

In [ ]:
dfs = [pd.DataFrame(data, columns=[method, 'cavity']).assign(**{method: lambda df: df[method] / df[method].iloc[0]}) for method, data in results_2.items()]

results_2_df = reduce(pd.merge, dfs)[['cavity', *results_2.keys()]]

In [ ]:
results_2_df

In [ ]:
(
    altair.Chart(results_2_df).mark_circle(size=60, color='red').encode(
        x=altair.X('original', scale=scale),
        y=altair.Y('fast', scale=scale),
        tooltip=cols
    ).interactive() +
    altair.Chart(results_2_df).mark_circle(size=60, color='blue').encode(
        x=altair.X('original', scale=scale),
        y=altair.Y('histogram', scale=scale),
        tooltip=cols
    ).interactive()
)

The four kinase active sits are identified by the `origina`l and `fast` methods, with the `original` method again showing a much clearer separartion. The `histogram` method has ranked most of the SYK activae sites highest, with one exception, although the score differences are small in all cases.

These results show that the slower `original` method performs best, with the `fast` method next. The fastest method, `histogram`, is not performing well here. Thus the trade-off between speed and quality of results is very plain here. 